Autores: Leandro Beloti Kornelius (211020900) e Lucca Magalhães Boselli Couto (222011552)
Turma de IIA 2023/2
Professor: Dibio

Com intuito de treinar conceitos de aprendizagem de máquina, foi desenvolvido um sistema de recomendação de filmes usando o modelo Naive Bayes e K-Nearest Neighbors, ou KNN. 
Sistemas de recomendação permeiam as atividades contemporâneas estando presentes em marketplaces, anúncios, conteúdos, entre outros. 
Tais aplicações fazem uso da inteligência artificial e consideram diversos parâmetros para fornecer sugestões personalizadas ao usuário.
Iniciaremos o relatório abordando o Modelo Naive Bayes o qual é inspirado no teorema de Bayes que, em sua maioria, assume independência entre os dados. Neste modelo é calculada a probabilidade de uma instância pertencer a uma categorização com base em condições/parâmetros pré determinadas.
Em contrapartida, o KNN é um método de aprendizagem supervisionado que, nesse caso, encontra itens semelhantes ao fornecido baseando-se nos itens mais próximos. Para o desenvolvimento do projeto foi utilizado o cosseno para mensurar a distância devido à utilização de palavras nos aspectos relevantes da base de dados.
Em função do aprender ter um limite de dados a serem enviados, foi necessário retirar a base de dados. Nesse aspecto, é preciso ir ao link https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset/data e baixar os arquivos credits.csv, keywords.csv e movies_metadata.csv e os inserí-los no diretório principal do projeto para rodar os algoritmos desenvolvidos pela equipe.

Iniciaremos falando sobre o modelo de Naive Bayes. Primeiramente, inicializamos as bibliotecas que serão utilizadas durante a leitura da base de dados e na construção do sistema de recomendação utilizando Naive Bayes.

In [66]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

Nessa etapa iremos ler a base de dados utilizando a biblioteca Pandas. É importante se atentar ao caminho do arquivo csv para que a leitura ocorra de maneira satisfatoria.

In [87]:
data = pd.read_csv('movies_metadata.csv', low_memory=False)

Agora iremos tratar as colunas que iremos utilizar como parâmetro de aprendizado durante o processo para que não haja valores nulos.

In [88]:
data['overview'] = data['overview'].fillna('')
data['title'] = data['title'].fillna('')

Nesse momento escolheremos os atributos de treinamento, o atributo alvo e dividiremos os dados de treinamento e teste.

In [89]:
# Atributos de treinamento
X = data[['title', 'overview']]

# Atributos alvo
y = ['recommended' if vote_average >= 5 else 'not recommended' for vote_average in data['vote_average']]

# Dividindo os dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Nessa etapa do projeto iremos vetorizar as informações contidas nos parâmetros escolhidos e transformá-las em atributos com valores numéricos para o aprendizado de máquina. Nesse sentido, utilizando a biblioteca presente no código conseguimos vetorizar numericamente informações e avaliar suas relevâncias conforme os cálculos de TF-IDF. Diante disto, é removido palavras frequentes especificadas pelo arquivo stop_words e, por isso, são levadas ao aprendizado palavras contadas relevantes à categorização do filme.

In [90]:
# Inicializando o vetorizador TF-IDF para as Stop Words
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# Vetorizando os atributos de treinamento
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train['title'] + ' ' + X_train['overview'])
X_test_tfidf = tfidf_vectorizer.transform(X_test['title'] + ' ' + X_test['overview'])

Agora iremos inicializar, treinar e fazer as previsões do modelo de Naive Bayes.

In [91]:
# Inicializando o modelo Naive Bayes (MultinomialNB)
model = MultinomialNB()

# Treinando o modelo
model.fit(X_train_tfidf, y_train)

# Fazendo as previsões do modelo
predictions = model.predict(X_test_tfidf)

Por fim, iremos analisar as métricas do sistema utilizando o Classification Report, que possui todas as métricas necessárias.

In [92]:
# Classification Report
cr = classification_report(y_test, predictions)
print("=============================")
print("Classification Report:")
print("=============================")
print(cr)
print("=============================")

Classification Report:
                 precision    recall  f1-score   support

not recommended       0.67      0.00      0.00      3011
    recommended       0.78      1.00      0.88     10629

       accuracy                           0.78     13640
      macro avg       0.72      0.50      0.44     13640
   weighted avg       0.75      0.78      0.68     13640

Classification Report:
                 precision    recall  f1-score   support

not recommended       0.67      0.00      0.00      3011
    recommended       0.78      1.00      0.88     10629

       accuracy                           0.78     13640
      macro avg       0.72      0.50      0.44     13640
   weighted avg       0.75      0.78      0.68     13640

Classification Report:
                 precision    recall  f1-score   support

not recommended       0.67      0.00      0.00      3011
    recommended       0.78      1.00      0.88     10629

       accuracy                           0.78     13640
      macro

Agora, nessa etapa, iremos explicar como foi feito o sistema de recomendação utilizando o modelo KNN.

Como no modelo anterior, iremos inicilizar as bibliotecas necessárias durante a execução do projeto.

In [93]:
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import classification_report

Iremos ler as bases de dados utilizadas.

In [94]:
metadata = pd.read_csv('movies_metadata.csv', low_memory=False)
credits = pd.read_csv('credits.csv')
keywords = pd.read_csv('keywords.csv')

Faremos um pequeno tratamento das linhas com informações quebradas.

In [95]:
# Remove linhas com IDs ruins
metadata = metadata.drop([19730, 29503, 35587])

Nesse momento do código, os IDs serão convertidos para valores inteiros e utilizados para unir informações presentes em keywords e credits de forma coerente ao ID especificado para a base de dados principal.

In [96]:
# Conversão dos IDs
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
metadata['id'] = metadata['id'].astype('int')

# Unindo keyword e credits na base de dados principal
metadata = metadata.merge(credits, on='id')
metadata = metadata.merge(keywords, on='id')

Consideramos como informações relevantes para serem vetorizadas no modelo KNN o diretor, elenco, palavras-chaves do roteiro e gêneros do filme. 
Logo, as funções abaixo visam retirar tais informações relevantes da base de dados de forma a poderem ser utilizadas na vetorização e análises futuras. 
Em seguida ocorre uma padronização destas características removendo espaços e tornando as palavras minúsculas. 
Por fim, é realizada uma união de todos esses parâmetros em uma grande string que será considerada na vetorização e, por consequência, na mensuração de proximidade.

In [97]:
# Itera nos parâmetros relevantes para o algoritmo
features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(literal_eval)

# Define functions to extract director, cast, genres, and keywords
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        if len(names) > 3:
            names = names[:3]
        return names
    return []

# Define new director, cast, genres, and keywords features
metadata['director'] = metadata['crew'].apply(get_director)
features = ['cast', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(get_list)

# Função que remove espaços e deixa o conglomerado de informações minúsculo
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

# Aplica a padronização de limpeza dos dados à todos parâmetros
features = ['cast', 'keywords', 'director', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(clean_data)

# Une todos textos relevantes
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])
metadata['soup'] = metadata.apply(create_soup, axis=1)

Nessa etapa do projeto iremos vetorizar as informações contidas nos parâmetros escolhidos e transformá-las em atributos com valores numéricos para o aprendizado de máquina. Nesse sentido, utilizando a biblioteca presente no código conseguimos vetorizar numericamente a frequência de cada palavra. Diante disto, é removido palavras frequentes especificadas pelo arquivo stop_words e, por isso, são levadas ao aprendizado palavras contadas relevantes à categorização do filme.

In [98]:
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(metadata['soup'])

Agora, iniciaremos e treinaremos o modelo KNN. Para isso, especificamos a quantidade de vizinhos próximos sendo 11 para a tomada de decisão, pois temos uma base de dados muito grande e para que não ocorra o overfitting, que consiste no fato de que o algoritmo fica viciado com os dados de treinamento e, consequentemente, não consegue classificar novos dados de teste. Além disso, utilizamos o cosseno como métrica de aproximação de informações, pois estaremos utilizando dados de texto para recomendar novos elementos.

In [99]:
knn_model = NearestNeighbors(n_neighbors=11, algorithm='auto', metric='cosine')
knn_model.fit(count_matrix)

NearestNeighbors(metric='cosine', n_neighbors=11)

De forma a calcular a pontuação de cada filme, iremos utilizar uma média ponderada que leva em consideração a nota do filme (vote_average) e a quantidade de votos registrados (vote_count). Especificamos que o filme é corretamente recomendado quando tem um "score" superior à 60%.

In [100]:
C = metadata['vote_average'].mean()

# Calculando a quantidade mínima de votos necessárias
m = metadata['vote_count'].quantile(0.70)

# Função que calcula o "score" de cada filme 
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    return (v / (v + m) * R) + (m / (m + v) * C)

# Aplicando a função da média ponderada para calcular o "score" de todos os filmes
metadata['weighted_rating'] = metadata.apply(weighted_rating, axis=1)

# Definindo o limite da popularidade baseando-se no "score" obtido
popularity_threshold = metadata['weighted_rating'].quantile(0.60)

Nessa etapa validaremos as recomendações obtidas para o critério estabelecido previamente (possuir "score" superior à 0.6).

In [101]:
# Função para validar a superioridade do score
def get_ground_truth(title):
    movie_idx = metadata.index[metadata['title'] == title].tolist()[0]
    return metadata['weighted_rating'][movie_idx] > popularity_threshold

# Função que retorna a validação para cada titulo de recomendação obtido
def get_ground_truth_labels(recommendations):
    return [get_ground_truth(movie) for movie, _ in recommendations]

A função principal do programa, que provê as recomendações do título recebido, faz o processamento dos dados como especificado pelo modelo e vetoriza os aspectos relevantes. 
O processamento realizado é enviado ao modelo KNN estabelecido que encontra os vizinhos próximos incluindo o próprio filme. Com este retorno obtemos as recomendações que serão posteriormente mostradas ao usuário.

In [102]:
def get_knn_recommendations(title):
    count = CountVectorizer(stop_words='english')
    count_matrix = count.fit_transform(metadata['soup'])

    knn_model = NearestNeighbors(n_neighbors=11, algorithm='auto', metric='cosine')
    knn_model.fit(count_matrix)

    movie_idx = metadata.index[metadata['title'] == title].tolist()[0]

    # Processa e vetoriza informações do título recebido
    movie_vector = count.transform([metadata['soup'][movie_idx]])

    # Encontra os vizinhos próximos
    _, indices = knn_model.kneighbors(movie_vector, n_neighbors=11)

    # Especifica os índices dos vizinnhos
    movie_indices = indices.flatten()

    # Cria a lista de filmes recomendados com base nos índices obtidos
    recommendations = [(metadata['title'].iloc[i], i) for i in movie_indices if i != movie_idx]

    return recommendations

Podemos testar o modelo com dois filmes para analisar o modelo. Para testar com "Star Wars", basta comentar a linha contendo o filme "Toy Story" e descomentar a de baixo.

In [103]:
recommendations = get_knn_recommendations("Toy Story")
# recommendations = get_knn_recommendations("Star Wars")

Nessa parte do código iremos efetivamente obter as recomendações obtidas e as mesmas serão mostradas ao usuário.

In [104]:
print("=============================")
print("Recommended Movies using Knn:")
print("=============================")
for movie, index in recommendations:
    print(f"{index}\t{movie}")

Recommended Movies using Knn:
1175	The Empire Strikes Back
1188	Return of the Jedi
22889	Behind Enemy Lines
10157	Star Wars: Episode III - Revenge of the Sith
22120	Ender's Game
7982	The Last Starfighter
2534	Star Wars: Episode I - The Phantom Menace
685	Solo
26770	Star Wars: The Force Awakens
5292	Star Wars: Episode II - Attack of the Clones
Recommended Movies using Knn:
1175	The Empire Strikes Back
1188	Return of the Jedi
22889	Behind Enemy Lines
10157	Star Wars: Episode III - Revenge of the Sith
22120	Ender's Game
7982	The Last Starfighter
2534	Star Wars: Episode I - The Phantom Menace
685	Solo
26770	Star Wars: The Force Awakens
5292	Star Wars: Episode II - Attack of the Clones
Recommended Movies using Knn:
1175	The Empire Strikes Back
1188	Return of the Jedi
22889	Behind Enemy Lines
10157	Star Wars: Episode III - Revenge of the Sith
22120	Ender's Game
7982	The Last Starfighter
2534	Star Wars: Episode I - The Phantom Menace
685	Solo
26770	Star Wars: The Force Awakens
5292	Star Wars:

Neste momento aplicamos o critério de validação estabelecido anteriormente para verificar se as recomendações estão de acordo com o especificado.

In [105]:
ground_truth_labels = get_ground_truth_labels(recommendations)

predicted_labels = [1] * len(recommendations)

Por fim, iremos analisar as métricas do modelo.

In [106]:
cr = classification_report(ground_truth_labels, predicted_labels)

print("=============================")
print("Classification Report:")
print("=============================")
print(cr)
print("=============================")

Classification Report:
              precision    recall  f1-score   support

       False       0.00      0.00      0.00         1
        True       0.90      1.00      0.95         9

    accuracy                           0.90        10
   macro avg       0.45      0.50      0.47        10
weighted avg       0.81      0.90      0.85        10



C:\Users\leobe\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\leobe\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\leobe\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

Classification Report:
              precision    recall  f1-score   support

       False       0.00      0.00      0.00         1
        True       0.90      1.00      0.95         9

    accuracy                           0.90        10
   macro avg       0.45      0.50      0.47        10
weighted avg       0.81      0.90      0.85        10



C:\Users\leobe\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\leobe\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\leobe\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

Classification Report:
              precision    recall  f1-score   support

       False       0.00      0.00      0.00         1
        True       0.90      1.00      0.95         9

    accuracy                           0.90        10
   macro avg       0.45      0.50      0.47        10
weighted avg       0.81      0.90      0.85        10



C:\Users\leobe\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\leobe\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\leobe\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

Classification Report:
              precision    recall  f1-score   support

       False       0.00      0.00      0.00         1
        True       0.90      1.00      0.95         9

    accuracy                           0.90        10
   macro avg       0.45      0.50      0.47        10
weighted avg       0.81      0.90      0.85        10



C:\Users\leobe\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\leobe\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\leobe\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

Classification Report:
              precision    recall  f1-score   support

       False       0.00      0.00      0.00         1
        True       0.90      1.00      0.95         9

    accuracy                           0.90        10
   macro avg       0.45      0.50      0.47        10
weighted avg       0.81      0.90      0.85        10



C:\Users\leobe\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\leobe\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\leobe\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

Em suma, temos dois modelos de aprendizado de máquina que buscam recomendar filmes ao usuário da maneira mais satisfatória possível. Em ambos os modelos adquirimos resultados bons na precisão e acurácia dos algoritmos implementados.